Words that occur nearby frequently are more important than words that only appear once or twice. Yet words that are too frequent — ubiquitous, like "the" or "good" — are unimportant. How can we balance these two conflicting constraints?

In [59]:
import numpy as np
import os

# directory where documents are stored
documents_directory = os.path.expanduser('documents')

In [60]:
# gets
def get_unique_terms_from_document(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # split document into terms and convert to a set of unique terms
    words = text.split()
    return words

In [61]:
def get_all_unique_terms(directory):
    unique_terms = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            unique_terms.extend(get_unique_terms_from_document(file_path))

    unique_terms = set(unique_terms)

    return unique_terms

unique_terms = get_all_unique_terms(documents_directory)

Squash the raw frequency using log10. A word appearing 100 times in a document doesn’t make that word 100 times more likely to be relevant to the meaning of the document. We also need to do something special with counts of 0, since we can’t take the log of 0.

$$
\text{tf}_{t,d} =
\begin{cases}
1 + \log_{10}(\text{count}(t,d)) & \text{if } \text{count}(t,d) > 0 \\
0 & \text{otherwise}
\end{cases}
$$

In [62]:
# we need to get the counts of terms in each document....

# term_document_matrix = np.zeros((num_terms, num_documents), dtype=int)

# get number of files in /documents

# Count the number of files in the directory
num_documents = len([name for name in os.listdir(documents_directory) if os.path.isfile(os.path.join(documents_directory, name))])
num_terms = len(unique_terms)

#term_document_matrix = np.zeros((num_terms, num_documents), dtype=int)

#for term in document:
    

Number of files: 5
